In [1]:
import squidpy as sq
import scanpy as sc
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, isspmatrix_csr

/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/paramiko/transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,
/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/numpydoc/docscrape.py:449: UserWarning: potentially wrong underline length... 
Extended Summary 
---------- in 
Add a Points layer to the layer list. 
...
  warn(msg)
/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/numpydoc/docscrape.py:449: UserWarning: potentially wrong underline length... 
Extended Summary 
---------- in 
Add a Labels layer to the layer list. 
...
  warn(msg)
/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/numpydoc/docscrape.py:449: UserWarning: potentially wrong underline length... 
Extended Summary 
---------- in 
Add a Shapes layer to the layer list. 
...
  warn(msg)
/home/dbdimitrov/anaconda3/lib/python3.8/site-packages/numpydoc/docscrape.py:449: UserWarning: potentially wrong underline length... 
Extended Summary 
-------

In [2]:
import liana as li

In [3]:
from matplotlib.pyplot import hist

In [9]:
from liana.method._global_lr_pipe import _global_lr_pipe
from liana.method.sp._spatialdm import _get_ordered_matrix, _standardize_matrix

In [4]:
counts = pd.read_csv("data/counts_mat.csv")
weights = pd.read_csv("data/weight_mat.csv")

In [5]:
var = pd.DataFrame(counts[['Unnamed: 0']]).set_index('Unnamed: 0')
var.index.name = None

In [6]:
adata = sc.AnnData(X=csr_matrix(counts.drop(counts.columns[0], axis=1), dtype=np.float32).T, var=var)

In [7]:
adata.obsm['proximity'] = csr_matrix(weights)

In [12]:
resource = pd.DataFrame({"ligand":["Dnm1l", "Arrb1", "Igf2", "Dnm1l"], "receptor":["Gucy1b3", "Mtor", "Tuba1a", "Fam63b"]})

In [13]:
temp, lr_res, ligand_pos, receptor_pos = _global_lr_pipe(adata=adata,
                                                         resource=resource,
                                                         expr_prop=0.05,
                                                         use_raw=False,
                                                         verbose=True,
                                                         layer=None,
                                                         _key_cols=['ligand_complex', 'receptor_complex'],
                                                         _complex_cols=['ligand_means', 'receptor_means'],
                                                         _obms_keys=['proximity'],
                                                         resource_name=None
                                                         )

Using `.X`!
Converting mat to CSR format


In [14]:
lr_res

,interaction,ligand,receptor,ligand_complex,receptor_complex,ligand_means,ligand_props,receptor_means,receptor_props,prop_min
0,Dnm1l&Gucy1b3,Dnm1l,Gucy1b3,Dnm1l,Gucy1b3,2.682889,0.950000,3.117442,0.976923,0.950000
1,Dnm1l&Fam63b,Dnm1l,Fam63b,Dnm1l,Fam63b,2.682889,0.950000,1.781250,0.830769,0.830769
2,Arrb1&Mtor,Arrb1,Mtor,Arrb1,Mtor,2.665413,0.957692,2.298451,0.907692,0.907692
3,Igf2&Tuba1a,Igf2,Tuba1a,Igf2,Tuba1a,1.924077,0.892308,6.466562,1.000000,0.892308


Correlations

In [ ]:
ligand_mat = _get_ordered_matrix(temp.X, ligand_pos, lr_res.ligand)
receptor_mat = _get_ordered_matrix(temp.X, receptor_pos, lr_res.receptor)

In [ ]:
x, y = ligand_mat[0, :].A.flatten(), receptor_mat[0, :].A.flatten()

In [ ]:
from scipy.stats import spearmanr, pearsonr, pointbiserialr, kendalltau

In [ ]:
pearsonr(x, y)

One by One

In [ ]:
def wcor(x, y, weight):
    weight_sum = np.sum(weight)
    
    numerator = weight_sum * np.sum(weight * x * y) - np.sum(weight * x) * np.sum(weight * y)
    denominator_x = weight_sum * np.sum(weight * (x**2)) - np.sum(weight * x)**2
    denominator_y = weight_sum * np.sum(weight * (y**2)) - np.sum(weight * y)**2
    
    wcor = numerator / np.sqrt(denominator_x * denominator_y)
    
    return wcor

In [ ]:
adata.obsm['proximity']

In [ ]:
i = 0

In [ ]:
# all weights = 1 (i.e. pearsonr)
wcor(x, y, np.repeat(1, x.shape[0]))

In [ ]:
weight = adata.obsm['proximity']

In [ ]:
# calculate all local correlations
# technically, if we take i (a spot), x and y (e.g. genes),
# we go over all rows in w (i.e. every spot)
# and we re-calculate PC with the spatial weights 
# for the spots i to any other spot (j)
local_corrs = []
for i in range(weight.shape[0]):
    local_corrs.append(wcor(x, y, weight[i, :].A))
local_corrs = np.array(local_corrs)

In [ ]:
hist(local_corrs)

In [ ]:
local_corrs[0:5]

masked

In [72]:
import scipy.stats as stats

In [73]:
ligand_mat = _get_ordered_matrix(temp.X, ligand_pos, lr_res.ligand)
receptor_mat = _get_ordered_matrix(temp.X, receptor_pos, lr_res.receptor)
x, y = ligand_mat[0, :].A.flatten(), receptor_mat[0, :].A.flatten()

In [74]:
def wcor_ma(x, y, weight):
    weight_sum = np.sum(weight)
    
    n1 = weight_sum * np.dot(weight, x * y)
    n2 = np.dot(weight, x) * np.dot(weight, y)
    numerator = n1 - n2
    
    denominator_x = weight_sum * np.dot(weight, x**2) - np.dot(weight, x)**2
    denominator_y = weight_sum * np.dot(weight, y**2) - np.dot(weight, y)**2
    
    corr = numerator / np.sqrt(denominator_x * denominator_y)
    
    return corr

In [75]:
weight = adata.obsm['proximity'].A

In [76]:
# msk = np.isnan(weight)
msk = np.logical_not(weight>0).astype(np.int16)

In [77]:
weight = np.ma.masked_array(weight, mask=msk)

In [78]:
xy_n = len(x)

In [79]:
x_ma = np.reshape(np.repeat(x, xy_n), newshape=(xy_n, xy_n))
y_ma = np.reshape(np.repeat(y, xy_n), newshape=(xy_n, xy_n))

x_ma = np.ma.masked_array(x_ma, mask=msk.T, fill_value=np.nan).T
y_ma = np.ma.masked_array(y_ma, mask=msk.T, fill_value=np.nan).T

In [80]:
x_ma = np.ma.masked_array(stats.mstats.rankdata(x_ma, axis=1), msk)
y_ma = np.ma.masked_array(stats.mstats.rankdata(y_ma, axis=1), msk)

In [85]:
local_corrs = []
for i in range(weight.shape[0]):
    local_corrs.append(wcor_ma(x_ma[i, :], y_ma[i, :], weight[i, :]))
local_corrs = np.array(local_corrs)

In [93]:
local_corrs[0:5]

array([ 0.14309412,  0.05121768,  0.25515995, -0.07335744, -0.30798529])

In [105]:
def wcor_ma_vec(weight, x, y):
    weight_sum = np.sum(weight)
    
    n1 = weight_sum * np.dot(weight, x * y)
    n2 = np.dot(weight, x) * np.dot(weight, y)
    numerator = n1 - n2
    
    denominator_x = weight_sum * np.dot(weight, x**2) - np.dot(weight, x)**2
    denominator_y = weight_sum * np.dot(weight, y**2) - np.dot(weight, y)**2
    
    corr = numerator / np.sqrt(denominator_x * denominator_y)
    
    return corr

In [141]:
result = np.apply_along_axis(lambda w, x, y: wcor_ma(w, x, y), axis=1, arr=weight, x=x_ma, y=y_ma)

In [148]:
def wcor_ma_2Dweights_vectorized_with_mask(x, y, weight):
    x_masked = np.ma.array(x, mask = weight.mask)
    y_masked = np.ma.array(y, mask = weight.mask)
    numerator = np.ma.sum(weight, axis=1) * np.ma.sum(weight * x_masked * y_masked, axis=1) - np.ma.sum(weight * x_masked, axis=1) * np.ma.sum(weight * y_masked, axis=1)
    
    denominator_x = np.ma.sum(weight, axis=1) * np.ma.sum(weight * x_masked**2, axis=1) - np.ma.sum(weight * x_masked, axis=1)**2
    denominator_y = np.ma.sum(weight, axis=1) * np.ma.sum(weight * y_masked**2, axis=1) - np.ma.sum(weight * y_masked, axis=1)**2
    wcor = numerator / np.ma.sqrt(denominator_x * denominator_y)
    
    return wcor.data

In [149]:
wcor_ma_2Dweights_vectorized_with_mask(x_ma, y_ma, weight)

array([ 1.43094117e-01,  5.12176843e-02,  2.55159949e-01, -7.33574366e-02,
       -3.07985292e-01, -1.02216035e-01, -1.39744467e-01, -1.80062597e-01,
       -2.07501403e-01,  1.05940134e-03, -1.20739368e-01, -9.96016202e-02,
        1.42116000e-01,  2.45454775e-01,  3.18804011e-01, -9.18780746e-02,
        1.13869262e-01,  5.10605709e-01, -6.54328027e-04, -9.61228225e-02,
        8.14484790e-02, -1.25280036e-01, -5.44812077e-02,  9.03697124e-02,
        1.36173821e-02, -1.29646395e-01,  9.11031099e-02,  2.09629867e-02,
       -5.15903309e-01, -1.64407021e-01, -6.43307472e-02, -3.53961308e-03,
       -7.52505261e-02, -2.11863265e-01, -1.55859029e-01,  1.06481763e-01,
       -1.48631081e-01,  1.32627652e-01, -1.79798104e-01, -2.37704435e-01,
       -1.32265266e-01,  3.68056026e-02, -4.23005980e-01,  1.60880271e-01,
       -3.85151275e-01, -8.33032654e-04,  3.04441955e-01,  1.09246788e-01,
        5.89446566e-01,  5.34232563e-01, -3.06996592e-01, -1.58774990e-01,
        3.78709653e-01,  

In [142]:
result

masked_array(
  data=[[[1.8563998400418653, 0.9578353225128343, 1.1964108244682299,
          ..., --, --, 1.018786250744946],
         [--, 1.0762790358996812, 1.0552461193959062, ..., --, --,
          1.005783335875203],
         [1.8307869649133215, 1.0274049628368787, 1.2464688892833775,
          ..., --, --, 1.0279688465781507],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [1.8212707848406253, 1.0374673805785108, 1.1899297423745943,
          ..., --, --, 1.0403285159903184]],

        [[-0.0018994726238753701, 1.49944027645516, 1.486613168075623,
          ..., --, --, 0.8613331381317845],
         [--, 1.6849042610764604, 1.3111345983132747, ..., --, --,
          0.8503456400238297],
         [-0.0018573312855537185, 1.6083726698159255, 1.548838768134609,
          ..., --, --, 0.8690334859542935],
         ...,
         [--, --, --, ..., --, --, --],
         [--, --, --, ..., --, --, --],
         [-0.001821439483732

In [83]:
x_ma.shape

(260, 260)

In [59]:
local_corrs

array([ 1.43094117e-01,  5.12176843e-02,  2.55159949e-01, -7.33574366e-02,
       -3.07985292e-01, -1.02216035e-01, -1.39744467e-01, -1.80062597e-01,
       -2.07501403e-01,  1.05940134e-03, -1.20739368e-01, -9.96016202e-02,
        1.42116000e-01,  2.45454775e-01,  3.18804011e-01, -9.18780746e-02,
        1.13869262e-01,  5.10605709e-01, -6.54328027e-04, -9.61228225e-02,
        8.14484790e-02, -1.25280036e-01, -5.44812077e-02,  9.03697124e-02,
        1.36173821e-02, -1.29646395e-01,  9.11031099e-02,  2.09629867e-02,
       -5.15903309e-01, -1.64407021e-01, -6.43307472e-02, -3.53961308e-03,
       -7.52505261e-02, -2.11863265e-01, -1.55859029e-01,  1.06481763e-01,
       -1.48631081e-01,  1.32627652e-01, -1.79798104e-01, -2.37704435e-01,
       -1.32265266e-01,  3.68056026e-02, -4.23005980e-01,  1.60880271e-01,
       -3.85151275e-01, -8.33032654e-04,  3.04441955e-01,  1.09246788e-01,
        5.89446566e-01,  5.34232563e-01, -3.06996592e-01, -1.58774990e-01,
        3.78709653e-01,  

In [41]:
i = 0 

In [42]:
x, y, weight = x_ma[i, :], y_ma[i, :], weight[i, :]

In [ ]:
weight_sum = np.sum(weight)

n1 = weight_sum * np.sum(weight * x * y)
n2 = np.sum(weight * x) * np.sum(weight * y)
numerator = n1 - n2

denominator_x = weight_sum * np.sum(weight * (x**2)) - np.sum(weight * x)**2
denominator_y = weight_sum * np.sum(weight * (y**2)) - np.sum(weight * y)**2

wcor = numerator / np.sqrt(denominator_x * denominator_y)

In [ ]:
local_corrs[0:5]

In [ ]:
hist(local_corrs)

In [ ]:
x_ma[i, :]

In [ ]:
y_ma[i, :]

In [ ]:
hist(local_corrs)

In [ ]:
wcor(x_ma[i,:], y_ma[i, :], weight[i, :])

In [ ]:
local_corrs.shape

Vectorize

For one interaction at a time:

In [ ]:
x, y = ligand_mat[0, :].A.flatten(), receptor_mat[0, :].A.flatten()

In [ ]:
weight = adata.obsm["proximity"].A ## does it really need to be A?

In [ ]:
weight_sum = np.sum(weight, axis = 1).flatten()

In [ ]:
n1 = (weight_sum * np.sum(np.multiply(weight, x) * y, axis=1))
n1[0:5]

In [ ]:
n2 = np.sum(np.multiply(weight, x), axis=1) * np.sum(np.multiply(weight, y), axis=1)
n2[0:5]

In [ ]:
numerator = n1 - n2

In [ ]:
numerator[0:5]

In [ ]:
denominator_x = weight_sum * np.sum(np.multiply(weight, (x**2)), axis = 1) - np.sum(np.multiply(weight,  x), axis=1)**2
denominator_y = weight_sum * np.sum(np.multiply(weight, (y**2)), axis = 1) - np.sum(np.multiply(weight,  y), axis=1)**2

In [ ]:
denominator_x.shape

In [ ]:
denominator_x[0:5]

In [ ]:
denominator_y[0:5]

In [ ]:
local_corrs = numerator / np.sqrt(denominator_x * denominator_y)

In [ ]:
hist(local_corrs)

In [ ]:
def _calculate_local_correlations(x, y, weight):
    weight_sum = np.sum(weight, axis = 1).flatten()
    
    numerator = weight_sum * np.sum(np.multiply(weight, x) * y, axis=1) - np.sum(np.multiply(weight, x), axis=1) * np.sum(np.multiply(weight, y), axis=1)
    denominator_x = weight_sum * np.sum(np.multiply(weight, (x**2)), axis = 1) - np.sum(np.multiply(weight,  x), axis=1)**2
    denominator_y = weight_sum * np.sum(np.multiply(weight, (y**2)), axis = 1) - np.sum(np.multiply(weight,  y), axis=1)**2
    
    local_corrs = numerator / np.sqrt(denominator_x * denominator_y)
    
    return local_corrs

In [ ]:
x, y = ligand_mat[0, :].A.flatten(), receptor_mat[0, :].A.flatten()

In [ ]:
_calculate_local_correlations(x, y, weight)

In [ ]:
import scipy.stats as stats

In [ ]:
weight = adata.obsm["proximity"].A
weight_sum = np.sum(weight, axis = 1).flatten()

In [ ]:
msk = np.isnan(weight)
# msk = np.logical_not(weight>0).astype(np.int16)

In [ ]:
weight = np.ma.masked_array(weight, mask=msk)

In [ ]:
xy_n = len(x)

In [ ]:
x_ma = np.reshape(np.repeat(x, xy_n), newshape=(xy_n, xy_n))
y_ma = np.reshape(np.repeat(y, xy_n), newshape=(xy_n, xy_n))

x_ma = np.ma.masked_array(x_ma, mask=msk, fill_value=np.nan).T
y_ma = np.ma.masked_array(y_ma, mask=msk, fill_value=np.nan).T

In [ ]:
x_ma[0:5, 0:5]

In [ ]:
# x_ma = np.ma.masked_array(stats.mstats.rankdata(x_ma, axis=0), msk)
# y_ma = np.ma.masked_array(stats.mstats.rankdata(y_ma, axis=0), msk)

In [ ]:
# vector
n1 = (((x_ma * y_ma).dot(weight.T)) * weight_sum)
n2 = (x_ma.dot(weight.T)) * (y_ma.dot(weight.T))
numerator = n1 - n2

denominator_x = (weight_sum * (x_ma ** 2).dot(weight.T)) - (x_ma.dot(weight.T))**2
denominator_y = (weight_sum * (y_ma ** 2).dot(weight.T)) - (y_ma.dot(weight.T))**2

local_corrs = (numerator / np.sqrt(denominator_x * denominator_y))


In [ ]:
numerator

In [ ]:
denominator_y[0:5,0]

In [ ]:
# one by one
weight_sum = np.sum(weight, axis = 1).flatten()

numerator = weight_sum * np.sum(np.multiply(weight, x) * y, axis=1) - np.sum(np.multiply(weight, x), axis=1) * np.sum(np.multiply(weight, y), axis=1)
denominator_x = weight_sum * np.sum(np.multiply(weight, (x**2)), axis = 1) - np.sum(np.multiply(weight,  x), axis=1)**2
denominator_y = weight_sum * np.sum(np.multiply(weight, (y**2)), axis = 1) - np.sum(np.multiply(weight,  y), axis=1)**2

local_corrs = numerator / np.sqrt(denominator_x * denominator_y)

In [ ]:
denominator_y[0:5]

In [ ]:
denominator_x.shape

In [ ]:
local_corrs.shape

In [ ]:
local_corrs.shape

In [ ]:
np.max(local_corrs)

In [ ]:
(x_ma.dot(weight)) * (y_ma.dot(weight))

All local corrs for all interactions

In [ ]:
ligand_mat

In [ ]:
receptor_mat

In [ ]:
weight = adata.obsm["proximity"].A.T ## does it really need to be A?

In [ ]:
weight_sums = np.sum(weight, axis = 0).flatten()

In [ ]:
x_mat = ligand_mat.A.T
y_mat = receptor_mat.A.T

In [ ]:
from scipy.stats import rankdata

In [ ]:
# x_mat = rankdata(x_mat, axis=0)
# y_mat = rankdata(y_mat, axis=0)

In [ ]:
n1 = (((x_mat * y_mat).T @ weight) * weight_sums.T)

In [ ]:
n1[0:5, 0]

In [ ]:
n2 = (x_mat.T @ weight) * (y_mat.T @ weight)

In [ ]:
numerator = n1 - n2

In [ ]:
numerator[0,0:5]

In [ ]:
denominator_x = (weight_sums * (x_mat ** 2).T @ weight) - (x_mat.T @ weight)**2
denominator_y = (weight_sums * (y_mat ** 2).T @ weight) - (y_mat.T @ weight)**2

In [ ]:
denominator_x[0, 0:5]

In [ ]:
denominator_y[0, 0:5]

In [ ]:
local_corrs = (numerator / np.sqrt(denominator_x * denominator_y)).T

In [ ]:
hist(local_corrs[:,0])

In [ ]:
local_corrs[0:5,0]

Clean

In [ ]:
import squidpy as sq
import scanpy as sc
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix, isspmatrix_csr

In [ ]:
import liana as li

In [ ]:
from matplotlib.pyplot import hist

In [ ]:
from liana.method._global_lr_pipe import _global_lr_pipe
from liana.method.sp._spatialdm import _get_ordered_matrix, _standardize_matrix

In [ ]:
from scipy.stats import rankdata

In [ ]:
# toy data and toy resource
adata = sc.read_h5ad("test_spatial.h5ad")
resource = pd.DataFrame({"ligand":["Dnm1l", "Arrb1", "Igf2"],
                         "receptor":["Gucy1b3", "Mtor", "Tuba1a"]}
                        )

In [ ]:
temp, lr_res, ligand_pos, receptor_pos = _global_lr_pipe(adata=adata,
                                                         resource=resource,
                                                         expr_prop=0.05,
                                                         use_raw=False,
                                                         verbose=True,
                                                         layer=None,
                                                         _key_cols=['ligand_complex', 'receptor_complex'],
                                                         _complex_cols=['ligand_means', 'receptor_means'],
                                                         _obms_keys=['proximity'],
                                                         resource_name=None
                                                         )

In [ ]:
# ligand-receptor mats
ligand_mat = _get_ordered_matrix(temp.X, ligand_pos, lr_res.ligand)
receptor_mat = _get_ordered_matrix(temp.X, receptor_pos, lr_res.receptor)

In [ ]:
x_mat = ligand_mat.A
y_mat = receptor_mat.A

In [ ]:
dist = adata.obsm['proximity']

In [ ]:
def calculate_local_correlations(x_mat, y_mat, dist, method="pearson"):
    if method not in ["pearson", "spearman", "spearman_nz"]:
        raise ValueError("method must be one of 'pearson', 'spearman', 'spearman_nz'")
    
    weight = dist.A.T ## does it really need to be A?
    weight_sums = np.sum(weight, axis = 0).flatten()
        
    if method=="spearman":
        x_mat = rankdata(x_mat, axis=1)
        y_mat = rankdata(y_mat, axis=1)
    
    # standard pearson
    n1 = (((x_mat * y_mat).dot(weight)) * weight_sums)
    n2 = (x_mat.dot(weight)) * (y_mat.dot(weight))
    numerator = n1 - n2
    
    denominator_x = (weight_sums * (x_mat ** 2).dot(weight)) - (x_mat.dot(weight))**2
    denominator_y = (weight_sums * (y_mat ** 2).dot(weight)) - (y_mat.dot(weight))**2
    
    local_corrs = (numerator / np.sqrt(denominator_x * denominator_y))
    
    return local_corrs

In [ ]:
local_pc = calculate_local_correlations(x_mat, y_mat, dist, method="pearson")
local_sp = calculate_local_correlations(x_mat, y_mat, dist, method="spearman")

In [ ]:
hist(local_pc[0,:])

In [ ]:
weight = dist.A.T ## does it really need to be A?

In [ ]:
np.mean(x_mat.dot(weight))

In [ ]:
msk = np.int16(weight==0)

In [ ]:
weight2 = np.ma.masked_array(data = weight, mask=msk, fill_value=np.nan)

In [ ]:
x_mat2 = np.ma.masked_array(x_mat, mask=np.isnan(x_mat))

In [ ]:
x_mat.shape

In [ ]:
xx = np.ma.masked_array.dot(x_mat2, weight2)

In [ ]:
weight2 * 2

In [ ]:
np.mean(xx)

In [ ]:
hist(local_spnz[0,:])

In [ ]:
from scipy.stats import spearmanr, pearsonr

In [ ]:
pearsonr(local_spnz[0,:], local_sp[0,:])

local p-values

In [ ]:
from numpy import random
from tqdm import tqdm

In [ ]:
rng = random.default_rng()
n_perm = 1000
positive_only = True # remove this option?

In [ ]:
local_pc.shape

In [ ]:
def _get_local_permutation_pvals(x_mat, y_mat, truth, function, dist, n_perm, positive_only=True, **kwargs):
    xy_n = truth.shape[0]
    spot_n = truth.shape[1]
    
    # permutation cubes to be populated
    local_pvals = np.zeros((xy_n, spot_n))
    
    # shuffle the matrix
    for i in tqdm(range(n_perm)):
        _idx = rng.permutation(spot_n)
        perm_r = function(x_mat = x_mat[:, _idx], y_mat=y_mat, dist=dist, **kwargs)
        if positive_only:
            local_pvals += np.array(perm_r >= truth, dtype=int)
        else:
            local_pvals += (np.array(np.abs(perm_r) >= np.abs(truth), dtype=int))

    local_pvals = local_pvals / n_perm
    
    return local_pvals
    

In [ ]:
local_pvals = _get_local_permutation_pvals(x_mat, y_mat, local_pc, calculate_local_correlations, dist, n_perm, positive_only=True)

In [ ]:
local_pc.shape[1]

In [ ]:
local_pvals.T

In [ ]:
spot_n = pc.shape[1]  # n of 1:1 edges 
xy_n = pc.shape[0] # n of x-y pairs (e.g. lrs)

# permutation cubes to be populated
local_pvals = np.zeros((xy_n, spot_n))

In [ ]:

for i in tqdm(range(n_perm)):
    _idx = rng.permutation(x_mat.shape[0])
    perm_r = calculate_local_correlations(x_mat[_idx, :], y_mat, dist)
    if positive_only:
        local_pvals += np.array(perm_r >= pc, dtype=int)
    else:
        local_pvals += (np.array(np.abs(perm_r) >= np.abs(pc), dtype=int))

local_pvals = local_pvals / n_perm

In [ ]:
local_pvals.shape